## Generate Grammar Items

This section of the notebook generates reads through every single map and creates mapping from a string to a column.

In [1]:
from map_generation.generate_image import pre_process_map
from tqdm import tqdm_notebook
import queue
import json
import os

In [2]:
TARGET_DIR = 'grammar/grammar_values/'
LEVELS_DIR = 'levels/csv_cleaned_maps'

In [3]:
columns = {}

In [4]:
files = os.listdir(LEVELS_DIR)
for file in files:
    # read file
    f = open(os.path.join(LEVELS_DIR, file))
    lines = f.readlines()
    f.close()
    
    # convert lines into pre-processed
    for i in range(len(lines)):
        lines[i] = lines[i].strip()
    pre_process_map(lines)
    
    # convert lines to grammar
    for line in lines:
        line_str = ','.join(line)
        if line_str not in columns:
            columns[line_str] = 0
        
        columns[line_str] += 1

In [5]:
grammar = {}
grammar_reversed = {}

i = 0
for key in columns:
    grammar[i] = key
    grammar_reversed[key] = i
    i += 1

In [6]:
f = open(os.path.join(TARGET_DIR, 'grammar.json'), 'w')
f.write(json.dumps(grammar))
f.close()

f = open(os.path.join(TARGET_DIR, 'grammar_reversed.json'), 'w')
f.write(json.dumps(grammar_reversed))
f.close()

## Grammar Generator

This section will read through the maps again, but this time use the mappings created, above, to create a full grammar.

In [7]:
CLEANED_MAPS = 'levels/csv_cleaned_maps/'
MAX_SIZE = 5

In [8]:
grammars = [{} for i in range(MAX_SIZE)]

In [9]:
def convert_to_level_matrix(lvl_matrix):
    for i in range(len(lvl_matrix)):
        lvl_matrix[i] = str(grammar_reversed[lvl_matrix[i].strip()])

In [10]:
def convert_to_grammar(lvl_matrix, lvl_grammar, size):
    q = queue.Queue()
    for i in range(size):
        q.put('0')
    
    for col in lvl_matrix[size:]:
        grammar_value = ','.join(list(q.queue))
        
        if grammar_value not in lvl_grammar:
            lvl_grammar[grammar_value] = {}

        g_value = str(grammar_reversed[','.join(col)])
        if g_value not in lvl_grammar[grammar_value]:
            lvl_grammar[grammar_value][g_value] = 0
            
        lvl_grammar[grammar_value][g_value] += 1
        
        last_val = q.get()
        q.put(g_value)

In [11]:
files = os.listdir(CLEANED_MAPS)

for file in files:
    f = open(os.path.join(CLEANED_MAPS, file), 'r')
    lines = f.readlines()
    f.close()
    
    for i in range(len(lines)):
        lines[i] = lines[i].strip()
    pre_process_map(lines)
    
    for size in range(MAX_SIZE):
        convert_to_grammar(lines, grammars[size], size+1)

In [12]:
for i in tqdm_notebook(range(len(grammars))):
    size_g = grammars[i]
    for g in size_g:
        total = 0
        for key in size_g[g]:
            total += size_g[g][key]

        for key in size_g[g]:
            size_g[g][key] /= float(total)

In [13]:
for i in range(len(grammars)):
    f = open('grammar/grammar_values/%s_grammar.json' % (str(i + 1)), 'w')
    f.write(json.dumps(grammars[i]))
    f.close()